In [9]:
import pennylane as qml
import numpy as np
from scipy.linalg import eigvals

# 定义量子设备，支持状态向量模拟
dev = qml.device("lightning.gpu", wires=4)

n_qubits = 4                # Number of qubits
q_depth = 4                 # Depth of the quantum circuit (number of variational layers)

# 量子电路的各个门定义（包括你的层次结构）
def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates."""
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RX_layer(w):
    """Layer of parametrized qubit rotations around the x axis."""
    for idx, element in enumerate(w):
        qml.RX(element, wires=idx)

def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis."""
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)
        
def U3_layer(nqubits):
    """layer of single qubit u3 rotations.
    """
    params = np.random.uniform(0, 2*np.pi, (nqubits, 3))
    for idx in range(nqubits):
        qml.U3(params[idx][0], params[idx][1], params[idx][2], wires=idx)

def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT."""
    for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
        qml.CNOT(wires=[i, i + 1])

@qml.qnode(dev)
def quantum_state(q_input_features, q_weights_flat):
    """计算量子态"""
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)
    
    H_layer(n_qubits)  # 施加 Hadamard 门
    RY_layer(q_input_features)  # 施加 RX 旋转门
    
    for k in range(q_depth):
        U3_layer(n_qubits)  # 施加 U3 旋转门
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

    return qml.state()  # 只返回量子态

# # 定义 Meyer-Wallach 测度计算
# def meyer_wallach_measure(rho):
#     """根据密度矩阵计算 Meyer-Wallach 测度"""
#     # 计算密度矩阵的特征值
#     eigenvalues = eigvals(rho)
    
#     # Meyer-Wallach测度 = 1 - 特征值的平方和
#     mw_measure = 1 - np.sum(np.square(np.abs(eigenvalues)))
    
#     return mw_measure

# # 示例输入（可以根据实际情况修改）
# q_input_features = np.random.uniform(0, 2 * np.pi, n_qubits)  # 输入特征（例如：随机生成）
# q_weights_flat = np.random.uniform(0, 2 * np.pi, n_qubits * q_depth)  # 随机生成量子权重

# # 获取量子态
# statevector = quantum_state(q_input_features, q_weights_flat)

# # 计算量子态的密度矩阵
# rho = np.outer(statevector, np.conj(statevector))

# # 计算并打印 Meyer-Wallach 测度
# mw_value = meyer_wallach_measure(rho)
# print(f"Meyer-Wallach测度: {mw_value}")


import pennylane as qml
import numpy as np

def partial_trace_pure(state, keep_qubits, n_qubits):
    """
    计算纯态的约化密度矩阵（手动实现偏迹）
    
    参数:
        state (np.array): 量子态向量（2^n 维）
        keep_qubits (list): 要保留的量子比特索引（如 [0] 表示第一个量子比特）
        n_qubits (int): 总量子比特数
    
    返回:
        rho (np.array): 约化密度矩阵
    """
    # 将状态向量重塑为 (2,2,...,2) 的张量
    psi = state.reshape([2]*n_qubits)
    
    # 计算要缩并的量子比特索引
    trace_over = [i for i in range(n_qubits) if i not in keep_qubits]
    
    # 计算偏迹: sum_{trace_over} |psi><psi|
    rho = np.tensordot(psi, psi.conj(), axes=(trace_over, trace_over))
    
    # 重塑为矩阵形式
    dim = 2**len(keep_qubits)
    return rho.reshape((dim, dim))

def meyer_wallach_measurement(state_vector, n_qubits):
    """
    计算给定量子态的 Meyer-Wallach 测度 Q
    
    参数:
        state_vector (np.array): 量子态向量（2^n 维复数数组）
        n_qubits (int): 量子比特数
    
    返回:
        Q (float): Meyer-Wallach 测度
    """
    Q = 0.0
    
    for k in range(n_qubits):
        # 计算约化密度矩阵 ρ_k
        rho_k = partial_trace_pure(state_vector, [k], n_qubits)
        
        # 计算纯度 Tr(ρ_k^2)
        purity = np.trace(rho_k @ rho_k).real  # 确保是实数
        
        # 累加线性熵 1 - Tr(ρ_k^2)
        Q += (1 - purity)
    
    # 归一化: Q = (2/n) * sum(1 - Tr(ρ_k^2))
    Q *= 2 / n_qubits
    
    return Q

# 示例使用
if __name__ == "__main__":
    # 随机生成输入参数
    q_input_features = np.random.rand(n_qubits) * 2 * np.pi
    q_weights_flat = np.random.rand(q_depth * n_qubits) * 2 * np.pi
    
    # 获取量子态
    state = quantum_state(q_input_features, q_weights_flat)
    
    # 计算 Q
    Q = meyer_wallach_measurement(state, n_qubits)
    print(f"Meyer-Wallach measure Q = {Q:.4f}")



Meyer-Wallach measure Q = 0.9141


In [12]:
import pennylane as qml
import numpy as np
from scipy.linalg import eigvals

# 定义量子设备，支持状态向量模拟
dev = qml.device("default.qubit", wires=4)

n_qubits = 4                # Number of qubits
q_depth = 4                 # Depth of the quantum circuit (number of variational layers)

# 量子电路的各个门定义
def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates."""
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RX_layer(w):
    """Layer of parametrized qubit rotations around the x axis."""
    for idx, element in enumerate(w):
        qml.RX(element, wires=idx)

def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis."""
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT."""
    for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
        qml.CNOT(wires=[i, i + 1])

@qml.qnode(dev)
def quantum_state(q_input_features, q_weights_flat):
    """计算量子态"""
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)
    
    H_layer(n_qubits)  # 施加 Hadamard 门
    RX_layer(q_input_features)  # 施加 RX 旋转门
    
    for k in range(q_depth):
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

    return qml.state()  # 只返回量子态



def partial_trace_pure(state, keep_qubits, n_qubits):
    """
    计算纯态的约化密度矩阵（手动实现偏迹）
    
    参数:
        state (np.array): 量子态向量（2^n 维）
        keep_qubits (list): 要保留的量子比特索引（如 [0] 表示第一个量子比特）
        n_qubits (int): 总量子比特数
    
    返回:
        rho (np.array): 约化密度矩阵
    """
    # 将状态向量重塑为 (2,2,...,2) 的张量
    psi = state.reshape([2]*n_qubits)
    
    # 计算要缩并的量子比特索引
    trace_over = [i for i in range(n_qubits) if i not in keep_qubits]
    
    # 计算偏迹: sum_{trace_over} |psi><psi|
    rho = np.tensordot(psi, psi.conj(), axes=(trace_over, trace_over))
    
    # 重塑为矩阵形式
    dim = 2**len(keep_qubits)
    return rho.reshape((dim, dim))

def meyer_wallach_measurement(state_vector, n_qubits):
    """
    计算给定量子态的 Meyer-Wallach 测度 Q
    
    参数:
        state_vector (np.array): 量子态向量（2^n 维复数数组）
        n_qubits (int): 量子比特数
    
    返回:
        Q (float): Meyer-Wallach 测度
    """
    Q = 0.0
    
    for k in range(n_qubits):
        # 计算约化密度矩阵 ρ_k
        rho_k = partial_trace_pure(state_vector, [k], n_qubits)
        
        # 计算纯度 Tr(ρ_k^2)
        purity = np.trace(rho_k @ rho_k).real  # 确保是实数
        
        # 累加线性熵 1 - Tr(ρ_k^2)
        Q += (1 - purity)
    
    # 归一化: Q = (2/n) * sum(1 - Tr(ρ_k^2))
    Q *= 2 / n_qubits
    
    return Q

# 示例使用
if __name__ == "__main__":
    # 随机生成输入参数
    q_input_features = np.random.rand(n_qubits) * 2 * np.pi
    q_weights_flat = np.random.rand(q_depth * n_qubits) * 2 * np.pi
    
    # 获取量子态
    state = quantum_state(q_input_features, q_weights_flat)
    
    # 计算 Q
    Q = meyer_wallach_measurement(state, n_qubits)
    print(f"Meyer-Wallach measure Q = {Q:.4f}")


Meyer-Wallach measure Q = 0.5962


In [14]:
import pennylane as qml
import numpy as np
from scipy.linalg import eigvals

# 定义量子设备，支持状态向量模拟
dev = qml.device("default.qubit", wires=4)

n_qubits = 4                # Number of qubits
q_depth = 4                 # Depth of the quantum circuit (number of variational layers)

# 量子电路的各个门定义
def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates."""
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

def RX_layer(w):
    """Layer of parametrized qubit rotations around the x axis."""
    for idx, element in enumerate(w):
        qml.RX(element, wires=idx)

def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT."""
    for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
        qml.CNOT(wires=[i, i + 1])

@qml.qnode(dev)
def quantum_state(q_input_features, q_weights_flat):
    """计算量子态"""
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)  # 重新调整权重形状
    
    H_layer(4)  # 施加 Hadamard 门
    RX_layer(q_input_features)  # 施加 RX 旋转门
    
    entangling_layer(4)  # 添加 CNOT 纠缠层

    return qml.state()  # 只返回量子态



def partial_trace_pure(state, keep_qubits, n_qubits):
    """
    计算纯态的约化密度矩阵（手动实现偏迹）
    
    参数:
        state (np.array): 量子态向量（2^n 维）
        keep_qubits (list): 要保留的量子比特索引（如 [0] 表示第一个量子比特）
        n_qubits (int): 总量子比特数
    
    返回:
        rho (np.array): 约化密度矩阵
    """
    # 将状态向量重塑为 (2,2,...,2) 的张量
    psi = state.reshape([2]*n_qubits)
    
    # 计算要缩并的量子比特索引
    trace_over = [i for i in range(n_qubits) if i not in keep_qubits]
    
    # 计算偏迹: sum_{trace_over} |psi><psi|
    rho = np.tensordot(psi, psi.conj(), axes=(trace_over, trace_over))
    
    # 重塑为矩阵形式
    dim = 2**len(keep_qubits)
    return rho.reshape((dim, dim))

def meyer_wallach_measurement(state_vector, n_qubits):
    """
    计算给定量子态的 Meyer-Wallach 测度 Q
    
    参数:
        state_vector (np.array): 量子态向量（2^n 维复数数组）
        n_qubits (int): 量子比特数
    
    返回:
        Q (float): Meyer-Wallach 测度
    """
    Q = 0.0
    
    for k in range(n_qubits):
        # 计算约化密度矩阵 ρ_k
        rho_k = partial_trace_pure(state_vector, [k], n_qubits)
        
        # 计算纯度 Tr(ρ_k^2)
        purity = np.trace(rho_k @ rho_k).real  # 确保是实数
        
        # 累加线性熵 1 - Tr(ρ_k^2)
        Q += (1 - purity)
    
    # 归一化: Q = (2/n) * sum(1 - Tr(ρ_k^2))
    Q *= 2 / n_qubits
    
    return Q

# 示例使用
if __name__ == "__main__":
    # 随机生成输入参数
    q_input_features = np.random.rand(n_qubits) * 2 * np.pi
    q_weights_flat = np.random.rand(q_depth * n_qubits) * 2 * np.pi
    
    # 获取量子态
    state = quantum_state(q_input_features, q_weights_flat)
    
    # 计算 Q
    Q = meyer_wallach_measurement(state, n_qubits)
    print(f"Meyer-Wallach measure Q = {Q:.4f}")


Meyer-Wallach measure Q = 0.0000


In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from scipy.linalg import eigvals

# 创建一个包含 4 个量子比特的量子电路
n_qubits = 4
qc = QuantumCircuit(n_qubits)

# 添加 Hadamard 门
for qubit in range(n_qubits):
    qc.h(qubit)

# 添加 RX 旋转门
for qubit in range(n_qubits):
    qc.rx(np.random.uniform(0, 2 * np.pi), qubit)  # 随机生成角度

# 添加 CNOT 门
for qubit in range(0, n_qubits - 1, 2):
    qc.cx(qubit, qubit + 1)

# 可视化电路
qc.draw('mpl')

# 获取量子态的状态向量
statevector = Statevector.from_instruction(qc).data  # .data 提取纯NumPy数组

# 计算密度矩阵
rho = np.outer(statevector, np.conj(statevector))

# 计算 Meyer-Wallach 测度
def meyer_wallach_measure(rho):
    """根据密度矩阵计算 Meyer-Wallach 测度"""
    eigenvalues = eigvals(rho)
    mw_measure = 1 - np.sum(np.square(np.abs(eigenvalues)))
    return mw_measure

# 计算并打印 Meyer-Wallach 测度
mw_value = meyer_wallach_measure(rho)
print(f"Meyer-Wallach测度: {mw_value}")


AttributeError: module 'qiskit._accelerate' has no attribute 'convert_2q_block_matrix'

In [4]:
import numpy as np
import pennylane as qml
from scipy.linalg import eigvals

# 创建量子设备
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def quantum_circuit():
    # 添加 Hadamard 门
    for qubit in range(4):
        qml.Hadamard(wires=qubit)

    # 增加更多的 CNOT 门，增加量子纠缠
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[1, 3])

    # 返回量子态
    return qml.state()

# 运行量子电路并获取量子态
statevector = quantum_circuit()

# 计算密度矩阵
rho = np.outer(statevector, np.conj(statevector))

purity = np.trace(np.dot(rho, rho))
print("Purity:", purity)


# 计算 Meyer-Wallach 测度
def meyer_wallach_measure(rho):
    """根据密度矩阵计算 Meyer-Wallach 测度"""
    eigenvalues = eigvals(rho)
    
    # 过滤掉接近零的特征值
    eigenvalues = eigenvalues[np.abs(eigenvalues) > 1e-10]
    mw_measure = 1 - np.sum(np.square(np.abs(eigenvalues)))
    return mw_measure

# 计算并打印 Meyer-Wallach 测度
mw_value = meyer_wallach_measure(rho)
print(f"Meyer-Wallach测度: {mw_value}")


Purity: (0.9999999999999986+0j)
Meyer-Wallach测度: 1.5543122344752192e-15


In [6]:
import pennylane as qml
import numpy as np
from scipy.linalg import sqrtm

n_qubits = 4                # Number of qubits
q_depth = 4                 # Depth of the quantum circuit (number of variational layers)

def meyer_wallach_measurement(state_vector, n_qubits):
    """
    计算给定量子态的 Meyer-Wallach 测度 Q。
    
    参数:
        state_vector (np.array): 量子态向量（2^n 维复数数组）
        n_qubits (int): 量子比特数
    
    返回:
        Q (float): Meyer-Wallach 测度
    """
    Q = 0.0
    
    for k in range(n_qubits):
        # 计算约化密度矩阵 ρ_k (对除第 k 个量子比特外的所有自由度取偏迹)
        rho_k = qml.math.reduce_dm(state_vector, [k])
        
        # 计算纯度 Tr(ρ_k^2)
        purity = np.trace(rho_k @ rho_k).real  # 确保是实数
        
        # 累加线性熵 1 - Tr(ρ_k^2)
        Q += (1 - purity)
    
    # 归一化: Q = (2/n) * sum(1 - Tr(ρ_k^2))
    Q *= 2 / n_qubits
    
    return Q

# 示例：计算你的量子电路的 Meyer-Wallach 测度
if __name__ == "__main__":
    # 随机生成输入参数
    q_input_features = np.random.rand(n_qubits) * 2 * np.pi
    q_weights_flat = np.random.rand(q_depth * n_qubits) * 2 * np.pi
    
    # 获取量子态
    state = quantum_state(q_input_features, q_weights_flat)
    
    # 计算 Q
    Q = meyer_wallach_measurement(state, n_qubits)
    print(f"Meyer-Wallach measure Q = {Q:.4f}")

NameError: name 'quantum_state' is not defined